In [ ]:
import numpy as np
import pandas as pd
import os
import re
import ast

In [ ]:
def clean_text(in_text):
    
    # replace '\n' with ' '
    
    # replace . with ' '
    in_text = in_text.replace('.', ' ')
    
    # replace , with ' '
    in_text = in_text.replace(',', ' ')
    
    # replace ' with ' '
    in_text = in_text.replace("'", "")
    
    # replace " with ' '
    in_text = in_text.replace('"', ' ')
    
    # replace '  ' with ' '
    #in_text = in_text.replace('  ', ' ')
    
    in_text = in_text.replace("\r","")
    in_text = in_text.replace("\n","")
    
    # Make the text all upper case - upper()
    
    return in_text.upper()

In [ ]:
def Get_WER(Original_text, Transcribed_text):
    
    # Tokenise the original text into list of words [Orig_lst]
    Orig_lst = Original_text.upper().split()
    
    # Tokenise the transcribed text into list of words [Transcribed_lst/TL]
    Transcribed_lst = Transcribed_text.upper().split()
    
    # List of words for matches: matched_lst
    matched_lst = []
    
    print("Type of matched_lst: " + str(type(matched_lst)))

    # List of words for deletions: deleted_lst
    deleted_lst = []

    # List of words for new insertions: inserted_lst
    inserted_lst = []

    # For each word in Orig_lst(SL):
    for src_word in Orig_lst:

        ## If the word is found in TL:        
        if src_word in Transcribed_lst:
            #print(src_word)
            # a) Add the word in match_lst
            matched_lst.append(src_word)
            #print("found!")
        
            # b) Remove the word from transcribed_list
            Transcribed_lst.remove(src_word)
        
        else: #(if word is not found in TL)
            # a) Add the word in deleted_list
            deleted_lst.append(src_word)
            #print("not found!")
        
        
    for word in Transcribed_lst:
        inserted_lst.append(word)
        
    return matched_lst, inserted_lst, deleted_lst


In [ ]:
# Read "transcription_analysis.csv"
df_transcription = pd.read_csv("transcription_analysis.csv")

df_transcription['GCP_match_list_raw'] = df_transcription['GCP_match_list_raw'].astype(object)
df_transcription['GCP_insertion_list_raw'] = df_transcription['GCP_insertion_list_raw'].astype(object)
df_transcription['GCP_deletion_list_raw'] = df_transcription['GCP_deletion_list_raw'].astype(object)


df_transcription['Azure_match_list_raw'] = df_transcription['Azure_match_list_raw'].astype(object)
df_transcription['Azure_insertion_list_raw'] = df_transcription['Azure_insertion_list_raw'].astype(object)
df_transcription['Azure_deletion_list_raw'] = df_transcription['Azure_deletion_list_raw'].astype(object)


df_transcription['LabTwin_match_list_raw'] = df_transcription['LabTwin_match_list_raw'].astype(object)
df_transcription['LabTwin_insertion_list_raw'] = df_transcription['LabTwin_insertion_list_raw'].astype(object)
df_transcription['LabTwin_deletion_list_raw'] = df_transcription['LabTwin_deletion_list_raw'].astype(object)


# for each row in df_transcription
for index, row in df_transcription.iterrows():
    
    print("Index: " + str(index))
    
    ## Assign Orig_text = row['Human_Transcription']
    if (row['Human_Transcription'] == pd.np.nan):
        continue
        
    Orig_text = clean_text(row['Human_Transcription'])
      
    ## For GCP---------------------------------------------START----
    ## Assign transcribed_text = row['Google_Transcription']
    transcribed_text = row['Google_Transcription']
    transcribed_text = clean_text(transcribed_text)
    
   # print(type(transcribed_text))

    if(len(transcribed_text) <= 0 or len(Orig_text) <= 0):
        continue
    
    ## matched_lst, inserted_lst, deleted_lst = Get_WER(Orig_text, transcribed_text)
    matched_lst, inserted_lst, deleted_lst = Get_WER(Orig_text, transcribed_text)
    
    print(type(matched_lst))
    
    
    print(matched_lst)
    
    ## Save matched_lst to row['GCP_match_list_raw']    
    df_transcription.at[index, 'GCP_match_list_raw'] = matched_lst
    
    ## save inserted_lst to row['GCP_insertion_list_raw']
    df_transcription.at[index, 'GCP_insertion_list_raw'] = inserted_lst
    
    ## save deleted_lst to row['GCP_deletion_list_raw']
    df_transcription.at[index, 'GCP_deletion_list_raw'] = deleted_lst
    
    ## Add the counts
    df_transcription.at[index, 'GCP_match_count_raw'] = len(matched_lst)
    df_transcription.at[index, 'GCP_insert_count_raw'] = len(inserted_lst)
    df_transcription.at[index, 'GCP_delete_count_raw'] = len(deleted_lst)    
    ## For GCP---------------------------------------------END----
    
    

    ## For Azure---------------------------------------------START----
    ## Assign transcribed_text = row['Azure_Transcription']
    transcribed_text = row['Azure_Transcription']
    transcribed_text = clean_text(transcribed_text)
    
    if(len(transcribed_text) <= 0 or len(Orig_text) <= 0):
        continue
    
    ## matched_lst, inserted_lst, deleted_lst = Get_WER(Orig_text, transcribed_text)     
    matched_lst, inserted_lst, deleted_lst = Get_WER(Orig_text, transcribed_text)
    
    ## Save matched_lst to row['Azure_match_list_raw'] 
    df_transcription.at[index, 'Azure_match_list_raw'] = matched_lst
        
    ## save inserted_lst to row['Azure_insertion_list_raw']    
    df_transcription.at[index, 'Azure_insertion_list_raw'] = inserted_lst
    
    ## save deleted_lst to row['Azure_deletion_list_raw']
    df_transcription.at[index, 'Azure_deletion_list_raw'] = deleted_lst
    
    
    ## Add the counts
    df_transcription.at[index, 'Azure_match_count_raw'] = len(matched_lst)
    df_transcription.at[index, 'Azure_insert_count_raw'] = len(inserted_lst)
    df_transcription.at[index, 'Azure_delete_count_raw'] = len(deleted_lst)
    ## For Azure---------------------------------------------END----
    
    

In [ ]:
df_transcription.to_csv("matched_transcription_analysis.csv", index=False)